In [1]:
#package importing
#from pyspark import ml
from pyspark.sql.functions import *
#from spark_rapids_ml.clustering import KMeans
from cuml.cluster import HDBSCAN
from pyspark.sql.functions import concat_ws,col,lit, row_number, monotonically_increasing_id
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
import pickle as pkl
import pandas as pd
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
#from lenspy import DynamicPlot
import pickle
from datetime import datetime
from pyspark.sql.types import FloatType
#from cuml import DBSCAN as dbscan
from sklearn import preprocessing
from numpy import int64
#from sklearn.metrics import davies_bouldin_score
import findspark
#https://spark.apache.org/docs/1.2.2/ml-guide.html
#https://pyshark.com/davies-bouldin-index-for-k-means-clustering-evaluation-in-python/
#import torch
findspark.init()

In [2]:
!export PYSPARK_SUBMIT_ARGS="--master local[2] pyspark-shell"
!export JAVA_HOME="/usr/bin/java"

In [ ]:
WEATHER_ROOT="/mnt/e/datasets/NOAA_GLERL/"
#code for combining the NOAA weather Data into a single DF
weatherData=pd.read_csv(WEATHER_ROOT+'test2.csv')
weatherData2019=pd.read_csv(WEATHER_ROOT+'test2019_2.csv')
def daypad(day):
    return(str(int64(day)).zfill(3))
def timepad(time):
    return(str(int64(time)).zfill(4))
def dt_convert(year, dayOfYear,hhmm):
    return datetime.strptime(f"{year} {dayOfYear} {hhmm}", '%Y %j %H%M')

weatherData['UTC']=weatherData.apply(lambda x: timepad(x['UTC']),axis=1)
weatherData['DOY']=weatherData.apply(lambda x: daypad(x['DOY']),axis=1)
weatherData['Date']=weatherData.apply(lambda x: dt_convert(str(int64(x['Year'])), str(x['DOY']),str(x['UTC'])), axis=1)#(dt_convert(weatherData['Year'],weatherData['DOY'],weatherData['UTC']))

weatherData2019['UTC']=weatherData2019.apply(lambda x: timepad(x['UTC']),axis=1)
weatherData2019['DOY']=weatherData2019.apply(lambda x: daypad(x['DOY']),axis=1)
weatherData2019['Date']=weatherData2019.apply(lambda x: dt_convert(str(int64(x['Year'])), str(x['DOY']),str(x['UTC'])), axis=1)
weatherDta=weatherData.append(weatherData2019, ignore_index=True)

In [3]:
AOT_ROOT='/mnt/e/datasets/Chicago-AoT-dataset/AoT_Chicago.complete.2022-08-31/'
OUT_ROOT='/mnt/e/outputs/chicago_aot/'
#pull in dataset(s) and start spark session
#with open('pre-processed/AoT_dataset_0.pkl', 'rb') as fil:
#    pdf=pd.DataFrame(pkl.load(fil))
#spark=SparkSession.builder.getOrCreate()
spark = SparkSession.builder.master('local[*]').config("spark.driver.memory", "30g").config("spark.driver.maxResultSize","20g").appName('Chicago_AoT').getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "UTC")
#spark = SparkSession.builder.config("spark.driver.memory", "20g").getOrCreate()
spark.conf.set('spark.rapids.sql.enabled','true')
spark.conf.set('spark.rapids.memory.gpu.pooling.enabled','true')
df=spark.read.csv(AOT_ROOT+'data.csv',sep=',', header=True)
df_nodes=spark.read.csv(AOT_ROOT+'nodes.csv',sep=',', header=True)
df_sensors=spark.read.csv(AOT_ROOT+'sensors.csv',sep=',', header=True)
#add Lat/Lon to the main dataset
df_new = df.withColumn('index', monotonically_increasing_id()).join(df_nodes, df.node_id == df_nodes.node_id, 'left').orderBy('index').select('timestamp', df.node_id, 'subsystem', 'sensor', 'parameter', 'value_raw', 'value_hrf', 'lat', 'lon')


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/16 09:27:20 WARN Utils: Your hostname, RogueLeader, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/16 09:27:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/16 09:27:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
max_id_substring='001e061' #this is the portion of the node_id that is identical to every node
strip_sens=['chemsense', 'metsense', 'loadavg', 'mem', 'time', 'device', 'net_rx', 'net_tx', 'ping', 'media', 'modem', 'disk_used', 'disk_size', 'disk_used_ratio', 'service_active', 'plugins', 'wagman_fc', 'wagman_cu', 'wagman_enabled', 'wagman_vdc', 'wagman_hb', 'wagman_stopping', 'wagman_starting', 'wagman_killing', 'wagman_th', 'wagman_comm', 'wagman_uptime', 'image_detector']
sensor_name_truncation={'chemsense':'cs','alphasense':'as','metsense':'ms','plantower':'pt','audio':'aud','lightsense':'ls','wagman':'wg','microphone':'mic','image':'img'}
#truncate all names of subsystems and node ids for space
stripdf=df_new.filter(~col('sensor').isin(strip_sens))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "chemsense", 'cs').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "metsense", 'ms').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "lightsense", 'ls').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "alphasense", 'as').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "plantower", 'pt').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "audio", 'aud').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "wagman", 'wg').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "microphone", 'mic').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("subsystem", when(col("subsystem") == "image", 'img').otherwise(col("subsystem")))
stripdf=stripdf.withColumn("node_id",expr("substring(node_id, 8, 12)"))
#add UNIX timestamp instead of timestamp
stripdf = stripdf.withColumn('unixTime',unix_timestamp(to_timestamp('timestamp','yyyy/MM/dd HH:mm:ss')))

#convert data timestamps to numbers
#stripdf=stripdf.withColumn("value_hrf",stripdf["value_hrf"].cast('float'))
#stripdf=stripdf.withColumn("value_raw",stripdf["value_raw"].cast('int'))


In [9]:
def calcUnixTime(timestamp:str):
    from datetime import datetime
    import pytz
    #code from Bing 2025OCT116
    # Define the date, time, and timezone
    date_str = timestamp#"2025-10-16 15:30:00"  # Example date and time
    timezone_str = "UTC"#"America/New_York"  # Example timezone
    # Parse the date and time
    dt = datetime.strptime(date_str, "%Y/%m/%d %H:%M:%S")
    # Localize the datetime object to the specified timezone
    timezone = pytz.timezone(timezone_str)
    localized_dt = timezone.localize(dt)
    # Convert to UNIX timestamp
    unix_timestamp = int(localized_dt.timestamp())
    print("UNIX Timestamp:", unix_timestamp)
    return unix_timestamp

In [11]:
ans=calcUnixTime('2018/01/01 00:00:00')

UNIX Timestamp: 1514764800


In [12]:
print(spark.conf.get('spark.driver.maxResultSize'))
time_start=calcUnixTime('2018/01/01 00:00:00')
time_end=calcUnixTime('2018/03/01 00:00:00')
test=stripdf.filter((stripdf.unixTime >= time_start)& (stripdf.unixTime<=time_end))#.contains('2018/01/01'))
test=test.drop('timestamp')

20g
UNIX Timestamp: 1514764800
UNIX Timestamp: 1519862400


In [ ]:
testdf=test.toPandas()


In [ ]:
testdf.to_csv(OUT_ROOT+'aot_2018JAN-APR.csv')

In [ ]:
#Filter what we're looking for
#result=df.filter((df.node_id=='001e0610e532') & (df.sensor=='lps25h') & (df.subsystem=='chemsense') & (df.parameter=='temperature'))

#result=result.filter((result['value_hrf'] != np.nan) | (result['value_raw'] != np.nan))
#convert date string to unixtime#no longer needed KDM2023SEP13.  There are built-in functions in pyspark to do this.  see https://stackoverflow.com/questions/77107769/pyspark-udf-function-storing-incorrect-data-despite-function-producing-correct-r
#def timeswap(x:str):
#    print(x)
#    utime= datetime.timestamp(datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))
#    print(utime)
#    return utime

#timeUDF = spark.udf.register('timeUDF',timeswap,FloatType()) 



In [ ]:
def timeswap2(x):
    #utime= datetime.timestamp(datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))
    #timeval=datetime.utcfromtimestamp(x)
    timeval=datetime.strptime(x, "%Y/%m/%d %H:%M:%S")
    #print(x)
    #print(utime)
    return timeval
def configureTraces(pdf,weatherData):
    data=[]
    tracev=go.Scattergl(x=pdf['timestamp'], y=pdf['value_hrf'],name='001e0610e532', mode="markers",marker=dict(size=3, symbol='circle-open'))
    data.append(tracev)
    tracep=go.Scattergl(x=pdf['timestamp'], y=pdf['prediction'] ,mode="markers",name='cluster',marker=dict(size=3, color='red', symbol='circle-open'),fillcolor='red')
    tracew=go.Scattergl(x=weatherData['Date'], y=weatherData['AirTemp(c)'] ,mode="markers",name='NOAA Data',marker=dict(size=3, color='green', symbol='circle-open'),fillcolor='red')
    data.append(tracew)
    data.append(tracep)
    return data

In [ ]:
#configure for time component
assembler_t = VectorAssembler(inputCols=['unixTime','value_hrf','value_raw'], outputCol="features",handleInvalid="keep")
result_t = assembler_t.transform(result)
normalizer_t=Normalizer(inputCol='features', outputCol='normalized_features')
result_t=normalizer_t.transform(result_t)
pdft=result_t.toPandas()

In [ ]:
result_t.select('normalized_features').show()

In [ ]:
#note: it looks like spark's normalizer and pandas' min-max scaler produce different numbers  need to fix this
def print_pandas(dataframe_given):
    with pd.option_context('display.max_rows', None,'display.max_columns', None, 'expand_frame_repr', False, 'display.max_colwidth', None):
        print("Given dataframe name")
        print(dataframe_given)
print_pandas(result_t.first()['normalized_features'])
print(x_scaled)

In [ ]:
#configure for NO time component
assembler_nt = VectorAssembler(inputCols=['value_hrf','value_raw'], outputCol="features",handleInvalid="keep")
result_nt = assembler_nt.transform(result)
pdfnt=result_nt.toPandas()